In [1]:
#Librerias a utilizar
import requests
import re
from bs4 import BeautifulSoup
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd
from pandas.io.json import json_normalize
import json

# Web scraping

In [2]:
#creamos una lista vacia, es donde vamos a estar recibiendo los enlaces de las paginas que usaremos para el web scraping
#dentro de un while, reviso que exista la palabra https://open.spotify.com/user/
#si no aparece esa información, entonces termina el while
urls=[]
while True:
    entrada = input("Hola, introduce el link de tu enlace de perfil de spotify, puedes introducir cualquier otra cosa para salir")
    if 'https://open.spotify.com/user/' in entrada:
        urls.append(entrada)
    else:
        break

Hola, introduce el link de tu enlace de perfil de spotify, puedes introducir cualquier otra cosa para salir https://open.spotify.com/user/1225924949?si=oPIIX-G2ReqZXii66zVHSg
Hola, introduce el link de tu enlace de perfil de spotify, puedes introducir cualquier otra cosa para salir 


In [3]:
#ejecuto el requests de cada una de las urls que ingresamos y lo guardo en una lista llamada html, 
#la cual es todo el codigo que se ejecuta en la construccion de la pagina web
#es importante aclarar que las paginas son creadas dinamicamente, por lo que este request, obtiene el codigo del script que difiere
#respecto del que en un principio se leyó
html =[requests.get(url).content for url in urls]

In [27]:
#ejecuto la expresion regular que me va a traer los usuarios de los enlaces de usuarios, esta busca el signos de ? y se trae cualquier elemento alfanumerico
#find all crea una lista, asi que por eso extraigo el elemento 0
usrs =[re.findall(r'\w*(?=\?)', url) for url in urls]
print(usrs)
usuariosID = [elemento[0] for elemento in usrs]

[['1225924949', '']]


In [5]:
#aqui estamos separando de todo el contenido de las html exclusivamente las playlist de la lista html
playlists = [re.findall(r'\/playlist\/\w*', str(pagina)) for pagina in html]

In [6]:
#ejecuto beautifulSoup para todas las paginas que estan en la lista html
soups = [BeautifulSoup(pagina, 'lxml') for pagina in html]

In [7]:
#busco todas las etiquetas de span, que tengan en el atributo dir el valor auto, esto nos arroja 
#una lista con los nombres de las playlist y los nombres de usuario
az = [soup.find_all('span',{'dir':'auto'}) for soup in soups]

In [8]:
#estas listas son en donde se van a guardar los nombres de las playlist y los de usuario, extrayendolos de la lista llamada az
PlstName=[]
nombres = []
for etiquetas in az:
    limpio =[]
    for contenido in etiquetas:
        #convierto a string y elimino las etiquetas de span
        limpio.append(str(contenido).replace('<span dir="auto">', '').replace('</span>', ''))
        #distingo el nombre de usuario, del nombre de la playlist
    nombres.append(limpio[0])
    PlstName.append(limpio[1:])
print("Nombre Playlist", PlstName)
print("nombres", nombres)

Nombre Playlist [['My Shazam Tracks', 'Running', 'July 2019', 'June 2019', 'May 2019', 'April 2019', 'March 2019', 'February 2019', 'January 2019', 'December 2018', 'November 2018', 'October 2018', 'September 2018', 'August 2018', 'July 2018', 'June 2018', 'May 2018', 'Marzo-Abril 2018', 'February 2018', 'January 2018', 'June 2017', 'July 2017', 'August 2017', 'September 2017', 'October 2017', 'November 2017', 'December 2017', 'Your Top Songs 2016', 'Mood: Groovy', 'Mood: Florence', 'Mood: Kygo', 'Mood: México Hermoso', 'Mood: The Lumineers', "Mood: I should've been born in a different time period.", 'Mood: Sweet, Corny, Teary', 'Mood: The xx', 'Mood: TWOD', 'Mood: Heal', 'Mood: John', 'Mood: Tay', 'Mood: Shivers', 'Mood: Electric Dreams', 'Mood: Country', 'Mood: CMEO', 'Mood: Motown', 'Mood: London', 'Mood: Drive', 'Mood: Lit', 'Focus: When I Cry, I Need A Score', 'Focus: Emotion Picture Soundtrack', 'Focus: Wired', 'Focus: Composed', 'Focus: Rob', 'Workout', 'Clase Prueba 4', 'Clase 

In [9]:
#Estas tres variables forman el nucleo final del webscraping
#en listota estoy guardando los diccionarios
listota = []
#el diccionario me separará los siguientes campos 'Id_usr', 'Nombre_perfil', 'PlaylistName','Id_playlist' que son las listas por separado
diccionario = {}
#comprimido hace tuplas de cada uno de los valores que recibe, en este caso uso listas en el orden que requiero para el diccionario
comprimido = zip(usuariosID, nombres, PlstName,playlists)

In [10]:
#Realizo la iteracion de cada elemento de comprimido (llamado rar) y despues la longitud del indice 2 (que son las playlist de los usuarios)
#y con ese campo que tiene la misma longitud de los nombres de las playlists formo el diccionario
for rar in comprimido:
    for p in range(len(rar[2])):
        diccionario= {'Id_usr':rar[0], 'Nombre_perfil':rar[1], 'PlaylistName':rar[2][p] ,'Id_playlist':rar[3][p]}
        #si no hago una copia del diccionario, siempre me va a agregar el mismo, por hacer la referencia al que se encuentra en la memoria
        listota.append(diccionario.copy())


In [28]:
#creamos el dataframe final, con los diccionarios de la listota
scrapdf = pd.DataFrame(listota)
#reinicio los indices solo para el caso de que se necesite mostrar el dataframe del webscrap
scrapdf.reset_index()

,index,Id_playlist,Id_usr,Nombre_perfil,PlaylistName
0,0,/playlist/6bJATygNAMi0irPQfq27lt,1225924949,Alejandro Arroyo Yamin,My Shazam Tracks
1,1,/playlist/5iUqf1eHMYDeK3vQK2sHJG,1225924949,Alejandro Arroyo Yamin,Running
2,2,/playlist/5Gq8alUIEDMszpFii0Dl7g,1225924949,Alejandro Arroyo Yamin,July 2019
3,3,/playlist/2RIQt8hv83sNa96kJamzNW,1225924949,Alejandro Arroyo Yamin,June 2019
4,4,/playlist/4QEVK3TW5RuTx86cEXuT3g,1225924949,Alejandro Arroyo Yamin,May 2019
5,5,/playlist/28GMqBdKjqKo6m7LekpNjz,1225924949,Alejandro Arroyo Yamin,April 2019
6,6,/playlist/2ghNDAtIk7LcspA7VMdj2L,1225924949,Alejandro Arroyo Yamin,March 2019
7,7,/playlist/5UxseVD5Dks7rxgqgHlTbf,1225924949,Alejandro Arroyo Yamin,February 2019
8,8,/playlist/2rdM7cDqzS6EBuRO7uaIkl,1225924949,Alejandro Arroyo Yamin,January 2019
9,9,/playlist/41zxUkllPHiShoMfS6ZWuQ,1225924949,Alejandro Arroyo Yamin,December 2018


In [29]:
scrapdf.to_csv("./output/UsrPlay.csv", index = False)

# API Parsing

In [32]:
#variables para poder conectar a la API de spotifi, son provistos por developer spotify
CLIENT_ID = "2b7f57a2e7d74e43b0a0771a99fbb470"
CLIENT_SECRET = "219cfd4ed31349d5b3175e3526de4921"

#Autentificacion y uso de las variables de arriba
credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

#Creacion de los token de autenticación de spotify
token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)


In [33]:
#Creamos el diccionario base con el que comenzaremos a concatenar, esas llaves son las mismas que trae la api de spotify
apiDF = pd.DataFrame({'href':[], 'items':[], 'limit':[], 'next':[], 'offset':[], 'previous':[], 'total':[]})

In [34]:
#en esta lista, guardaremos los dataframes que resulten de la iteracion del metodo .user_playlist_tracks
dataframes = []

In [35]:
#usamos el id de usuario y el id de la playlist, que proviene del data frame del resultado anterior 
#para obtener las canciones y todos los datos que se manejan en la bd de spotify y lo guardo en una lista
for row in scrapdf.itertuples():
    canciones = spotify.user_playlist_tracks(row.Id_usr, str(row.Id_playlist).replace('/playlist/', ''))
    dataframes.append(pd.DataFrame(canciones))

In [36]:
#hacemos concat, para agregar todos los dataframes guardados en la lista a un data drafe mas grande
apiDF = pd.concat(dataframes)

In [37]:
#solo compruebo que se conservan los mismos nombres de las columnas del dataframe llamado apiDF
apiDF.columns
#reinicio los indices, para que sean continuos, si no se hiciera, no se mostraria el indice correcto
apiDF.reset_index()

,index,href,items,limit,next,offset,previous,total
0,0,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
1,1,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
2,2,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
3,3,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
4,4,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
5,5,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
6,6,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
7,7,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
8,8,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164
9,9,https://api.spotify.com/v1/playlists/6bJATygNA...,"{'added_at': '2017-11-29T20:21:42Z', 'added_by...",100,https://api.spotify.com/v1/playlists/6bJATygNA...,0,None,164


In [38]:
#en candionero2, lo que hago es normalizar el json de la columna llamada items y a continuacion la imprimo solo para ver cuales son las columnas
#que se encuentran dentro de items
candionero2 = json_normalize(apiDF['items'])
print(candionero2.columns)
#creo un dataframe conformado por las columnas de candionero2, con los campos que me sirven
dfinal = candionero2[['added_by.id','track.name', 'track.external_urls.spotify', 'track.popularity']]

Index(['added_at', 'added_by.external_urls.spotify', 'added_by.href',
       'added_by.id', 'added_by.type', 'added_by.uri', 'is_local',
       'primary_color', 'track.album.album_type', 'track.album.artists',
       'track.album.available_markets', 'track.album.external_urls.spotify',
       'track.album.href', 'track.album.id', 'track.album.images',
       'track.album.name', 'track.album.release_date',
       'track.album.release_date_precision', 'track.album.total_tracks',
       'track.album.type', 'track.album.uri', 'track.artists',
       'track.available_markets', 'track.disc_number', 'track.duration_ms',
       'track.episode', 'track.explicit', 'track.external_ids.isrc',
       'track.external_urls.spotify', 'track.href', 'track.id',
       'track.is_local', 'track.name', 'track.popularity', 'track.preview_url',
       'track.track', 'track.track_number', 'track.type', 'track.uri',
       'video_thumbnail.url'],
      dtype='object')


In [39]:
#guardamos el dataframe en un csv
dfinal.to_csv("./output/SpotifyApi.csv", index = False)